In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# to split the datasets
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# load the Titanic Dataset with a few variables for demonstration

data = pd.read_csv("/content/drive/MyDrive/Feature Engineering/Datasets/titanic.csv", usecols=["age", "fare", "survived"])
data.head()

,survived,age,fare
0,1,29.0000,211.3375
1,1,0.9167,151.5500
2,0,2.0000,151.5500
3,0,30.0000,151.5500
4,0,25.0000,151.5500


In [4]:
# let's look at the percentage of NA

data.isnull().mean()

survived    0.000000
age         0.200917
fare        0.000764
dtype: float64

In [5]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    data[["age", "fare"]],  # predictors
    data["survived"],  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0,
)  # seed to ensure reproducibility

X_train.shape, X_test.shape

((916, 2), (393, 2))

In [6]:
# Let's explore the missing data in the train set
# the percentages should be fairly similar to those
# of the whole dataset

X_train.isnull().mean()

age     0.191048
fare    0.000000
dtype: float64

In [7]:
# add the missing indicator

# this is done very simply by using np.where from numpy
# and isnull from pandas:

X_train["Age_NA"] = np.where(X_train["age"].isnull(), 1, 0)
X_test["Age_NA"] = np.where(X_test["age"].isnull(), 1, 0)

X_train.head()

,age,fare,Age_NA
501,13.0,19.5000,0
588,4.0,23.0000,0
402,30.0,13.8583,0
1193,NaN,7.7250,1
686,22.0,7.7250,0


In [8]:
# the mean of the binary variable, coincides with the
# perentage of missing values in the original variable

X_train["Age_NA"].mean()

0.19104803493449782

In [9]:
# yet the original variable, still shows the missing values
# which need to be replaced by any of the techniques
# we have learnt

X_train.isnull().mean()

age       0.191048
fare      0.000000
Age_NA    0.000000
dtype: float64

In [10]:
# for example median imputation

median = X_train["age"].median()

X_train["age"] = X_train["age"].fillna(median)
X_test["age"] = X_test["age"].fillna(median)

# check that there are no more missing values
X_train.isnull().mean()

age       0.0
fare      0.0
Age_NA    0.0
dtype: float64

In [11]:
# we are going to use the following variables,
# some are categorical some are numerical

cols_to_use = [
    "LotFrontage",
    "MasVnrArea",  # numerical
    "BsmtQual",
    "FireplaceQu",  # categorical
    "SalePrice",  # target
]


In [12]:
# let's load the House Prices dataset

data = pd.read_csv("/content/drive/MyDrive/Feature Engineering/Datasets/houseprice.csv", usecols=cols_to_use)
print(data.shape)
data.head()

(1460, 5)


,LotFrontage,MasVnrArea,BsmtQual,FireplaceQu,SalePrice
0,65.0,196.0,Gd,NaN,208500
1,80.0,0.0,Gd,TA,181500
2,68.0,162.0,Gd,TA,223500
3,60.0,0.0,TA,Gd,140000
4,84.0,350.0,Gd,TA,250000


In [13]:
# let's inspect the variables with missing values

data.isnull().mean()

LotFrontage    0.177397
MasVnrArea     0.005479
BsmtQual       0.025342
FireplaceQu    0.472603
SalePrice      0.000000
dtype: float64

In [14]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    data.drop("SalePrice", axis=1),
    data["SalePrice"],
    test_size=0.3,
    random_state=0,
)

X_train.shape, X_test.shape

((1022, 4), (438, 4))

In [15]:
# Remove target from variable list

cols_to_use = cols_to_use[:-1]

cols_to_use

['LotFrontage', 'MasVnrArea', 'BsmtQual', 'FireplaceQu']

In [16]:
# Let's create a list with the name
# of the new variables

indicators = [f"{var}_NA" for var in cols_to_use]

In [17]:
# Let's add the indicators
X_train[indicators] = X_train[cols_to_use].isna().astype(int)
X_test[indicators] = X_test[cols_to_use].isna().astype(int)

In [18]:
# Let's explore the first rows

X_train.head()

,LotFrontage,MasVnrArea,BsmtQual,FireplaceQu,LotFrontage_NA,MasVnrArea_NA,BsmtQual_NA,FireplaceQu_NA
64,NaN,573.0,Gd,NaN,1,0,0,1
682,NaN,0.0,Gd,Gd,1,0,0,0
960,50.0,0.0,TA,NaN,0,0,0,1
1384,60.0,0.0,TA,NaN,0,0,0,1
1100,60.0,0.0,TA,NaN,0,0,0,1


In [19]:
# now let's evaluate the mean value of the missing indicators

# calculate the mean:

X_train[indicators].mean()

LotFrontage_NA    0.184932
MasVnrArea_NA     0.004892
BsmtQual_NA       0.023483
FireplaceQu_NA    0.467710
dtype: float64

In [20]:
# the mean of the missing indicator
# coincides with the percentage of missing values
# in the original variable

X_train.isnull().mean()

LotFrontage       0.184932
MasVnrArea        0.004892
BsmtQual          0.023483
FireplaceQu       0.467710
LotFrontage_NA    0.000000
MasVnrArea_NA     0.000000
BsmtQual_NA       0.000000
FireplaceQu_NA    0.000000
dtype: float64

In [21]:
# let's impute the NA with  the median for numerical
# variables
# remember that we calculate the median using the train set

median = X_train["LotFrontage"].median()
X_train["LotFrontage"] = X_train["LotFrontage"].fillna(median)
X_test["LotFrontage"] = X_test["LotFrontage"].fillna(median)

median = X_train["MasVnrArea"].median()
X_train["MasVnrArea"] = X_train["MasVnrArea"].fillna(median)
X_test["MasVnrArea"] = X_test["MasVnrArea"].fillna(median)


# let's impute the NA in categorical variables by the
# most frequent category (aka the mode)
# the mode needs to be learnt from the train set

mode = X_train["BsmtQual"].mode()[0]
X_train["BsmtQual"] = X_train["BsmtQual"].fillna(mode)
X_test["BsmtQual"] = X_test["BsmtQual"].fillna(mode)

mode = X_train["FireplaceQu"].mode()[0]
X_train["FireplaceQu"] = X_train["FireplaceQu"].fillna(mode)
X_test["FireplaceQu"] = X_test["FireplaceQu"].fillna(mode)

In [22]:
# and now let's check there are no more NA
X_train.isnull().mean()

LotFrontage       0.0
MasVnrArea        0.0
BsmtQual          0.0
FireplaceQu       0.0
LotFrontage_NA    0.0
MasVnrArea_NA     0.0
BsmtQual_NA       0.0
FireplaceQu_NA    0.0
dtype: float64